```{contents}
```
## **Delegation in LangGraph**

**Delegation** in LangGraph is the architectural mechanism by which one node or agent **assigns responsibility for a subtask to another agent or subgraph**, then **integrates the returned result into the global execution state**.
It enables **modular design, scalable reasoning, parallel execution, and multi-agent coordination**.

---

### **1. Motivation for Delegation**

Large problems cannot be solved efficiently by a single reasoning process.
Delegation enables:

| Need               | Benefit                                         |
| ------------------ | ----------------------------------------------- |
| Task decomposition | Break complex problems into simpler subproblems |
| Specialization     | Assign tasks to expert agents                   |
| Parallelism        | Execute subtasks concurrently                   |
| Fault isolation    | Failures localized to subgraphs                 |
| Scalability        | Systems grow without exponential complexity     |

In practice, delegation transforms a graph from a pipeline into a **distributed problem-solving system**.

---

### **2. Delegation Mental Model**

```
Supervisor Agent
       |
       |  delegates
       ↓
Worker Agent A  → returns result
Worker Agent B  → returns result
Worker Agent C  → returns result
       |
       ↓
Supervisor integrates & decides
```

The supervisor controls **when, what, and how** tasks are delegated.

---

### **3. Delegation via Subgraphs**

In LangGraph, delegation is implemented using **subgraphs** or **agent nodes**.

#### **Subgraph Delegation**

```python
research_graph = StateGraph(ResearchState)
research_graph.add_node("researcher", research_node)
research_graph.set_entry_point("researcher")
research_graph.add_edge("researcher", END)

research = research_graph.compile()
```

The main graph delegates:

```python
def supervisor(state):
    result = research.invoke({"topic": state["topic"]})
    return {"research": result["notes"]}
```

This is **hard delegation**: full control transfer until completion.

---

### **4. Delegation via Agent Nodes**

```python
def supervisor(state):
    if state["task"] == "math":
        return {"route": "math_agent"}
    else:
        return {"route": "writing_agent"}
```

```python
builder.add_conditional_edges(
    "supervisor",
    lambda s: s["route"],
    {
        "math_agent": "math",
        "writing_agent": "writer"
    }
)
```

This is **dynamic delegation** decided at runtime.

---

### **5. State Contract in Delegation**

Delegation is safe only if **state interfaces** are enforced.

| Rule              | Purpose                         |
| ----------------- | ------------------------------- |
| Input contract    | Subgraph knows what it receives |
| Output contract   | Supervisor knows what to expect |
| Partial updates   | Workers modify only their scope |
| Reducer functions | Merge safely into global state  |

```python
class GlobalState(TypedDict):
    topic: str
    research: str
    draft: str
```

---

### **6. Delegation Patterns**

| Pattern           | Description                                 |
| ----------------- | ------------------------------------------- |
| Supervisor–Worker | Central controller assigns tasks            |
| Planner–Executor  | Planner delegates execution                 |
| Expert Routing    | Task routed to domain expert                |
| Map–Reduce        | Many workers → one aggregator               |
| Swarm             | Agents coordinate without central authority |

---

### **7. Production Example: Supervisor–Worker System**

```python
def supervisor(state):
    return {"task": "research"}

def researcher(state):
    return {"research": "Findings..."}

def writer(state):
    return {"draft": "Article..."}

builder = StateGraph(GlobalState)

builder.add_node("supervisor", supervisor)
builder.add_node("researcher", researcher)
builder.add_node("writer", writer)

builder.set_entry_point("supervisor")

builder.add_conditional_edges(
    "supervisor",
    lambda s: s["task"],
    {"research": "researcher", "write": "writer"}
)

builder.add_edge("researcher", "supervisor")
builder.add_edge("writer", END)
```

This forms a **delegation loop** controlled by the supervisor.

---

### **8. Delegation vs Traditional Pipelines**

| Pipeline              | Delegation                |
| --------------------- | ------------------------- |
| Fixed sequence        | Dynamic routing           |
| Single reasoning path | Multiple reasoning agents |
| Hard-coded flow       | State-driven flow         |
| Low scalability       | Highly scalable           |

---

### **9. Engineering Guarantees**

| Property        | Why It Matters                      |
| --------------- | ----------------------------------- |
| Isolation       | Workers cannot corrupt global state |
| Observability   | Each delegation step is logged      |
| Fault tolerance | Supervisor can retry delegation     |
| Scalability     | Workers can be parallelized         |
| Governance      | Human approval can gate delegation  |

---

### **10. When to Use Delegation**

Use delegation when:

* Tasks have **clear substructure**
* Domains require **specialized reasoning**
* Work can be **parallelized**
* System must remain **extensible**

---

### **11. Conceptual Summary**

> Delegation in LangGraph turns a single LLM workflow into a **distributed cognitive system**
> where specialized components collaborate through **explicit state contracts and controlled execution**.



### Demonstration

In [1]:
from typing import TypedDict
from langgraph.graph import StateGraph, END

# ------------------ State Schema ------------------

class GlobalState(TypedDict):
    topic: str
    task: str
    research: str
    draft: str

# ------------------ Worker Agents ------------------

def researcher(state: GlobalState):
    print("🔍 Researcher working...")
    return {"research": f"Key facts about {state['topic']}"}

def writer(state: GlobalState):
    print("✍️ Writer working...")
    return {"draft": f"Article on {state['topic']} using {state['research']}"}

# ------------------ Supervisor ------------------

def supervisor(state: GlobalState):
    if not state.get("research"):
        return {"task": "research"}
    if not state.get("draft"):
        return {"task": "write"}
    return {"task": "done"}

# ------------------ Graph Construction ------------------

builder = StateGraph(GlobalState)

builder.add_node("supervisor", supervisor)
builder.add_node("researcher", researcher)
builder.add_node("writer", writer)

builder.set_entry_point("supervisor")

builder.add_conditional_edges(
    "supervisor",
    lambda s: s["task"],
    {
        "research": "researcher",
        "write": "writer",
        "done": END
    }
)

builder.add_edge("researcher", "supervisor")
builder.add_edge("writer", "supervisor")

graph = builder.compile()

# ------------------ Run ------------------

result = graph.invoke({
    "topic": "LangGraph Delegation",
    "task": "",
    "research": "",
    "draft": ""
})

print("\nFinal State:")
print(result)


🔍 Researcher working...
✍️ Writer working...

Final State:
{'topic': 'LangGraph Delegation', 'task': 'done', 'research': 'Key facts about LangGraph Delegation', 'draft': 'Article on LangGraph Delegation using Key facts about LangGraph Delegation'}
